# Fashion MNIST Keras


- Tensorflow搭建model
- 使用Keras进行BP
- 可视化

In [29]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [17]:
# 导入数据集
(x,y),(x_t,y_t) = datasets.fashion_mnist.load_data()
x.shape

(60000, 28, 28)

In [7]:
x_t.shape

(10000, 28, 28)

In [50]:
# 制作数据集
batch_size = 128
def pre_process(x,y):
    x = tf.cast(x,dtype=tf.float32)
    x = tf.convert_to_tensor(x,dtype=tf.float32) /255.
    y = tf.cast(y,dtype=tf.int32)
    return x,y

db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(pre_process).shuffle(5000).batch(batch_size)
db_t = tf.data.Dataset.from_tensor_slices((x_t,y_t))
db_t = db_t.map(pre_process).shuffle(5000).batch(batch_size)

db_iter = iter(db)

In [21]:
# 搭建网络

model = Sequential([
    layers.Dense(256,activation=tf.nn.relu) ,
    layers.Dense(128,activation=tf.nn.relu) ,
    layers.Dense(64,activation=tf.nn.relu) ,
    layers.Dense(32,activation=tf.nn.relu) ,
    layers.Dense(10)
])
model.build(input_shape=[None,28*28])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [74]:
# 训练

def train(db,db_test,max_iter):
    opt = optimizers.Adam()
    for i in range(max_iter):
        for j,(x,y) in enumerate(db):
            x = tf.reshape(x,[-1,28*28])
            with tf.GradientTape() as tape:
                hidd = model(x)
                y_one_hot = tf.one_hot(y,depth=10)
                loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_one_hot,hidd,from_logits = True))
            grads = tape.gradient(loss,model.trainable_variables)
            opt.apply_gradients(zip(grads,model.trainable_variables))
            if j%100 == 0:
                print(i,'-',j,'--> loss = ',float(loss))
        # validation
        cor_num = 0
        num = 0
        for  x,y in db_test:
            x = tf.reshape(x,[-1,28*28]) 
            prob = tf.nn.softmax(model(x),axis=1)    # b , 10
            pred = tf.argmax(prob,axis=1)
            pred = tf.cast(pred,dtype=tf.int32)
            corr = tf.equal(y,pred)
            cor_num += tf.reduce_sum(tf.cast(corr,dtype=tf.int32))
            num += x.shape[0]
        print('acc = %.6f' % float(cor_num/num))
            

In [75]:
train(db,db_t,3)

0 - 0 --> loss =  0.08530659228563309
0 - 100 --> loss =  0.17189857363700867
0 - 200 --> loss =  0.10539513826370239
0 - 300 --> loss =  0.11625084280967712
0 - 400 --> loss =  0.10358424484729767
acc = 0.884500
1 - 0 --> loss =  0.049116261303424835
1 - 100 --> loss =  0.12599235773086548
1 - 200 --> loss =  0.10951802879571915
1 - 300 --> loss =  0.133076012134552
1 - 400 --> loss =  0.09060138463973999
acc = 0.882600
2 - 0 --> loss =  0.06419236958026886
2 - 100 --> loss =  0.1400873064994812
2 - 200 --> loss =  0.08423995971679688
2 - 300 --> loss =  0.16700324416160583
2 - 400 --> loss =  0.10230924189090729
acc = 0.885500


### 可视化


搭配tensorflow的可视化工具：Tensorboard

工作原理：
    - 监听tensorflow在磁盘记录的log
    - 建立监听实例（可远程监听）
`
tensorboard --logdir logs
`

步骤：
1. 在文件夹下开启tensorboard
`
tensorboard --logdir logs
`

2. 复制网址（端口号6006）
*注意：localhost:6006没用，需要复制控制台显示的网址*

3. 在py文件中添加tensorboard代码
   - tf.summary.create_file_writer(file_name)  file_name 通常是时间
   - 在训练的for循环中添加监控变量或图片
   ```
       with summary_writer.as_default():
           tf.summary.scalar('loss',float(loss),step = j)
   ```